In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

%matplotlib inline
sns.set(style="darkgrid")
pd.set_option('display.float_format', lambda x: '%.2f' % x)
warnings.filterwarnings("ignore")

In [ ]:
df_sample = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
df_sample[0:5]

In [ ]:
df_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')

In [ ]:
df_train[0:5]

In [ ]:
df_train.shape

In [ ]:
df_shop =pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
df_shop[0:5]

In [ ]:
df_item = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')

In [ ]:
df_item[0:5]

In [ ]:
df_test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
df_test.tail()

In [ ]:
df_test[0:5]

In [ ]:
df_item_categories =pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')

In [ ]:
df_item_categories[0:5]

In [ ]:
df_shop.isnull().values.any()

merging df_train and df_shop on shop id

In [ ]:
df_train = df_train.merge(df_shop,on = ['shop_id'])

In [ ]:
df_train[0:5]

In [ ]:
df_shop['shop_name'][df_shop['shop_id'] == 59]

merging df_train and df_item on item id

In [ ]:
df_train = df_train.merge(df_item,on = ['item_id'])

In [ ]:
df_train[0:5]

In [ ]:
df_item[df_item['item_id'] == 22154]

merging df_train and df_item_categories on item_category_id

In [ ]:
df_train = df_train.merge(df_item_categories,on =['item_category_id'])

In [ ]:
df_train[0:5]

In [ ]:
df_item_categories[df_item_categories['item_category_id'] == 37]

In [ ]:
df_train.isnull().values.any()

In [ ]:
df_train.describe()

In [ ]:
df_train.date = df_train.date.apply(lambda x:datetime.datetime.strptime(x, '%d.%m.%Y'))

In [ ]:
df_train['date'].describe()

In [ ]:
df_train['date'].min().date()

In [ ]:
df_train['date'].max().date()

## data cleaning

### item_price

In [ ]:
# the skewed box plot shows us the presence of outliers
%matplotlib notebook
%matplotlib inline
sns.boxplot(y="item_price", data =df_train)
plt.show()

In [ ]:
#calculating 0-100th percentile to find a the correct percentile value for removal of outliers
for i in range(0,100,10):
    var = df_train["item_price"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
#looking further from the 99th percecntile
for i in range(90,100):
    var = df_train["item_price"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
#calculating speed values at each percntile 99.0,99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100
for i in np.arange(0.0, 1.0, 0.1):
    var =df_train["item_price"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(99+i,var[int(len(var)*(float(99+i)/100))]))
print("100 percentile value is ",var[-1])

In [ ]:
plt.plot(var[-5:])
plt.show()

In [ ]:
plt.plot(var[-2:])
plt.show()

* so we can set the thresold of 50000

### item_cnt_day

In [ ]:
# the skewed box plot shows us the presence of outliers
%matplotlib notebook
%matplotlib inline
sns.boxplot(y="item_cnt_day", data =df_train)
plt.show()

In [ ]:
#calculating 0-100th percentile to find a the correct percentile value for removal of outliers
for i in range(0,100,10):
    var = df_train["item_cnt_day"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
#looking further from the 99th percecntile
for i in range(90,100):
    var = df_train["item_cnt_day"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(i,var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
#calculating speed values at each percntile 99.0,99.1,99.2,99.3,99.4,99.5,99.6,99.7,99.8,99.9,100
for i in np.arange(0.0, 1.0, 0.1):
    var =df_train["item_cnt_day"].values
    var = np.sort(var,axis = None)
    print("{} percentile value is {}".format(99+i,var[int(len(var)*(float(99+i)/100))]))
print("100 percentile value is ",var[-1])

In [ ]:
plt.plot(var[-20:])
plt.show()

In [ ]:
plt.plot(var[-3:])
plt.show()

setting thresold 750

In [ ]:
df_train = df_train[df_train['item_price']<50000]

In [ ]:
df_train.shape

In [ ]:
df_train = df_train[df_train['item_cnt_day']<750]

In [ ]:
df_train = df_train[df_train['item_cnt_day']>=0]

In [ ]:
df_train.shape

In [ ]:
print('Data set size before filter valid:', df_train.shape)
# Only shops that exist in test set.
df_train = df_train[df_train['shop_id'].isin(df_test['shop_id'].unique())]
# Only items that exist in test set.
df_train = df_train[df_train['item_id'].isin(df_test['item_id'].unique())]
print('Data set size after filter valid:', df_train.shape)

In [ ]:
df_train[0:5]

conclusion : the shops that are in test data are all present in train data,but train data doesnot consist of all the item that are present in the test data. So we have to fill the missing values.

In [ ]:
# Aggregate to monthly level the sales
df_train_groupby= df_train.groupby(["date_block_num","shop_id","item_id"])[
    "item_price","item_cnt_day"].agg({"item_price":"mean","item_cnt_day":"sum"})

In [ ]:
df_train_groupby = df_train_groupby.reset_index()

In [ ]:
df_train_groupby[0:5]

In [ ]:
df_train_groupby.shape

In [ ]:
a =  list(df_train_groupby.item_cnt_day)

In [ ]:
a

In [ ]:
df_test[0:5]

In [ ]:
df_test.shape

In [ ]:
final_data = pd.merge(df_test,df_train_groupby,on = ['item_id','shop_id'],how = 'left')

In [ ]:
final_data[0:5]

In [ ]:
final_data.shape

In [ ]:
final_data.isnull().values.any()

In [ ]:
final_data.fillna(0,inplace = True)

In [ ]:
final_data.isnull().values.any()

In [ ]:
final_data.drop(['shop_id','item_id'],inplace = True, axis = 1)


In [ ]:
final_data[0:5]

In [ ]:
#We will create pivot table.
# Rows = each shop+item code
# Columns will be out time sequence
pivot_data = final_data.pivot_table(index='ID',values = 'item_cnt_day' ,columns='date_block_num',fill_value = 0 )

In [ ]:
pivot_data.head()

In [ ]:
row = 34

In [ ]:
pivot_data.values[0:1][0][row-3-1: row -1]

In [ ]:
sum(a[25:])

In [ ]:
a  =(pivot_data[0:1].values)[0]

## now we have predict the sale of the item related to the id for november month

### now using simple moving average

In [ ]:
def simple_average(pivot_data,row_no,col_no,window_size):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        #print(temp[col_no-window_size-1:col_no-1])
        predict_value = int(sum(temp[col_no-window_size-1:col_no-1])/window_size)
        #print(predict_value)
        predicted_values.append(predict_value)
        
    return predicted_values

In [ ]:
def actual_values(pivot_data):
    actual = []
    temp = pivot_data.shape[0]
    for i in range(temp):
        temp = pivot_data.values[i:i+1][0]
        actual.extend([int(temp[-1:])])
    return actual

In [ ]:
actual = actual_values(pivot_data)

In [ ]:
def rms_error(predicted,actual):
    final =   list(np.array(actual) - np.array(store))
    mse = sum([e**2 for e in final])/len(final)
    return mse

In [ ]:
# here we have predicted the october item_cnt using the previous three months
error_values = []
window_lenght = 6
for i in range(2,window_lenght):
    store = simple_average(pivot_data,pivot_data.shape[0],pivot_data.shape[1],i)
    error = rms_error(store,actual)
    error_values.extend([error])

In [ ]:
# here we have use window lenght from 2 to 5
print(error_values)

In [ ]:
def predict_nov_simple_average(pivot_data,row_no,col_no,window_size):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        #print(temp[col_no-window_size:col_no])
        predict_value = int(sum(temp[col_no-window_size:col_no])/window_size)
        #print(predict_value)
        predicted_values.append(predict_value)
        
    return predicted_values

In [ ]:
# we are getting the best result from window_size of 3 so we will use window_size of 3 to predict value for november data
predicted_nov_savg = predict_nov_simple_average(pivot_data,pivot_data.shape[0],pivot_data.shape[1],3)

In [ ]:
len(predicted_nov_savg)

In [ ]:
predicted_nov_savg

### using weighted average to predict prediction for novrmber month

In [ ]:
def weighted_average(pivot_data,row_no,col_no,window_size):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        #print(temp[col_no-window_size-1:col_no-1])
        temp = temp[col_no-window_size-1:col_no-1]
        store = 0
        for j in range(1,window_size+1):
            store = store + j*temp[j-1]
        predict_value = int(store/((window_size*(window_size+1))/2))
        #print(predict_value)
        predicted_values.append(predict_value)
        
    return predicted_values

In [ ]:
# here we have predicted the october item_cnt using the previous three months
error_values = []
window_lenght = 6
for i in range(2,window_lenght):
    store = weighted_average(pivot_data,pivot_data.shape[0],pivot_data.shape[1],i)
    error = rms_error(store,actual)
    error_values.extend([error])

In [ ]:
error_values

In [ ]:
#here the window size of 4 gives the best result so we will use window size of 4 to predict the item_cnt for month november
def predict_nov_weighted_average(pivot_data,row_no,col_no,window_size):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        temp = temp[col_no-window_size:col_no]
        #print(temp[col_no-window_size:col_no])
        store =0
        for j in range(1,window_size+1):
            store = store + j*temp[j-1]
        predict_value = int(store/((window_size*(window_size+1))/2))
        #print(predict_value)
        predicted_values.append(predict_value)
        
    return predicted_values

In [ ]:
predicted_nov_weighted_average = predict_nov_weighted_average(pivot_data,pivot_data.shape[0],pivot_data.shape[1],4)

In [ ]:
len(predicted_nov_weighted_average)

In [ ]:
predicted_nov_weighted_average

### using exponential average to predict item_cnt for november month

In [ ]:
def exponential_average(pivot_data,row_no,col_no,alpha):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        #print(temp[:-1])
        temp = temp[:-1]
        store = 0
        for j in range(len(temp)):
            store = store*(1-alpha) + alpha*temp[j]
        #print(predict_value)
        store = int(store)
        predicted_values.append(store)
        
    return predicted_values

In [ ]:
# here we have predicted the october item_cnt using the previous three months
error_values = []
window_lenght = [0.3,0.4,0.5,0.6,0.7]
for i in window_lenght:
    store = exponential_average(pivot_data,pivot_data.shape[0],pivot_data.shape[1],i)
    error = rms_error(store,actual)
    error_values.extend([error])

In [ ]:
error_values

In [ ]:
def exponential_average_nov(pivot_data,row_no,col_no,alpha):
    predicted_values = []
    for i in range(row_no):
        temp = pivot_data.values[i:i+1][0]
        #print(temp[:-1])
        #temp = temp[:-1]
        store = 0
        for j in range(len(temp)):
            store = store*(1-alpha) + alpha*temp[j]
        #print(predict_value)
        store = int(store)
        predicted_values.append(store)
        
    return predicted_values

In [ ]:
predicted_nov_exponential_average = exponential_average_nov(pivot_data,pivot_data.shape[0],pivot_data.shape[1],0.4)

In [ ]:
len(predicted_nov_exponential_average)

### now using autoregressive model

In [ ]:
# one thing to consider while autoregressive model is that the value of lag means how many previous values you are going to use
# we can plot autocorelation to find out the value of lag.
# here we are plotting the curve for id 0
fig, axs = plt.subplots(2, 2)
axs[0,0].plot(pivot_data.values[0:1][0])
axs[0,1].plot(pivot_data.values[1:2][0])

axs[1,0].plot(pivot_data.values[2:3][0])
axs[1,1].plot(pivot_data.values[3:4][0])

#plt.plot(series)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
#fig, axs = plt.subplots(2, 2)

plot_acf(pivot_data.values[0:1][0],lags = 33)
plot_acf(pivot_data.values[1:2][0],lags = 33)
plot_acf(pivot_data.values[2:3][0],lags = 33)
plot_acf(pivot_data.values[3:4][0],lags = 33)

plt.show()

In [ ]:
def auto_regressive(pivot_data,row_no,lag):
    predicted_values = []
    for i in range(row_no):
        data = pivot_data.values[i:i+1][0]
        #print(data[:-1])
        model = AutoReg(data[:-1],lags = lag).fit()
        #print(model.params)
        predict_value = model.predict(start=len(data)-1,end=len(data)-1)
        #print(predict_value)
        predicted_values.append(int(predict_value))  
        
    return predicted_values

In [ ]:
'''def auto_regressive_2(pivot_data,row_no,lag):
    predicted_values = []
    for i in range(row_no):
        data = pivot_data.values[i:i+1][0]
        data = data[:-1]
        window = len(data)
        #print(window)
        model = AutoReg(data[:-1],lags = lag).fit()
        parameters = model.params
        yhat = parameters[0]
        for j in range(lag):
            yhat += parameters[j+1] * data[window-j-1]
        #print(model.params)
        #print(yhat)
        #predict_value = model.predict(start=len(data)-1,end=len(data)-1)
        #print(predict_value)
        predicted_values.append(int(yhat))  
        
    return predicted_values'''

In [ ]:
'''store = auto_regressive_2(pivot_data,pivot_data.shape[0],4)'''

In [ ]:
'''print(rms_error(store,actual))'''

In [ ]:
# here we have multiple time series for each id 
# we will fit model on series one by one 
# and then check oveall mean squared error
from statsmodels.tsa.ar_model import AutoReg

error_values = []
lag = [2,3,4,5]
for i in lag:
    store = auto_regressive(pivot_data,pivot_data.shape[0],i)
    error = rms_error(store,actual)
    error_values.extend([error])


In [ ]:
# the result are not good while using auto regressive model
error_values

### now using lstm model for prediction

In [ ]:
# X we will keep all columns execpt the last one 
X_train = np.expand_dims(pivot_data.values[:,:-1],axis = 2)
# the last column is our prediction
y_train = pivot_data.values[:,-1:]

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
# for test we keep all the columns execpt the first one
X_test = np.expand_dims(pivot_data.values[:,1:],axis = 2)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from keras.models import load_model, Model

# our defining sales model 
model = Sequential()
model.add(LSTM(units = 64,input_shape = (33,1)))
#sales_model.add(LSTM(units = 64,activation='relu'))
model.add(Dense(128))
model.add(Dropout(0.3))
model.add(Dense(64))
model.add(Dropout(0.2))

model.add(Dense(1))


In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_squared_error'])
model.fit(X_train,y_train,batch_size = 300,epochs = 10)

In [ ]:
output_november_sales = model.predict(X_test)

In [ ]:
output_november_sales[0:10]